In [1]:
### same as main.py but as a jpynb

import open3d as o3d 
import numpy as np
import trimesh

import datato3D as d3d


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Bounding Boxes

In [2]:

path = 'testfiles\Muensterhuegel.txt'
bbox_product = d3d.read_lv95_do_rectangular_bbox(path)
#
print(bbox_product)

POLYGON ((2611400 1267200, 2611650 1267200, 2611650 1267450, 2611400 1267450, 2611400 1267200))


In [3]:
bbox_working_region = d3d.buffer_polygon(bbox_product, 0.1)
#
print(bbox_working_region)

POLYGON ((2611300 1267200, 2611300 1267450, 2611300.4815273327 1267459.801714033, 2611301.9214719594 1267469.5090322017, 2611304.3059664266 1267479.0284677255, 2611307.612046749 1267488.2683432365, 2611311.8078735652 1267497.1396736826, 2611316.8530387697 1267505.557023302, 2611322.6989546637 1267513.4393284163, 2611329.2893218813 1267520.7106781187, 2611336.5606715837 1267527.3010453363, 2611344.442976698 1267533.1469612303, 2611352.8603263176 1267538.1921264348, 2611361.7316567637 1267542.3879532511, 2611370.9715322745 1267545.6940335731, 2611380.4909677985 1267548.0785280403, 2611390.198285967 1267549.5184726673, 2611400 1267550, 2611650 1267550, 2611659.801714033 1267549.5184726673, 2611669.5090322015 1267548.0785280403, 2611679.0284677255 1267545.6940335731, 2611688.2683432363 1267542.3879532511, 2611697.1396736824 1267538.1921264348, 2611705.557023302 1267533.1469612303, 2611713.4393284163 1267527.3010453363, 2611720.7106781187 1267520.7106781187, 2611727.3010453363 1267513.43932

# DEM


In [4]:
### if your DEM-file does not cover the area, try d3d.download_raster_files() and stitch_raster_files() to get a DEM that covers the area
### automatically download and stich raster files from the csv you get when you draw a polygon on https://www.swisstopo.admin.ch/de/hoehenmodell-swissalti3d

# csv_path = "example_Belinzona/ch.swisstopo.swissalti3d-ix4nQqFB.csv"
# output_directory = "example_Belinzona/"
# raster_files = d3d.download_raster_files(csv_path, output_directory)

# output_filename = "example_Belinzona/merged_raster.tif"
# d3d.stitch_raster_files(raster_files, output_filename)

In [5]:

dem_path = 'testfiles\DOM_Punktgitter_2021_50cm.tif'
out_image, out_transform, src = d3d.read_raster_dem_cut_to_bbox(dem_path, bbox_working_region)
out_image = d3d.slice_out_image(out_image)
#
print(out_image.shape)

(901, 901)


In [6]:
# The parameters are (out_image, out_transform, resolution, max_triangle_size, max_triangle_height)
mesh, pcd = d3d.dem_to_mesh(out_image, out_transform, 0.1, 30, 8)

# Visualize the mesh and pointcloud
o3d.visualization.draw_geometries([mesh], mesh_show_wireframe=True, mesh_show_back_face=True)


In [7]:
mesh_cropped = d3d.cutting_mesh_with_bbox(mesh, bbox_product, 100, 300)
# Visualize the cut mesh and pointcloud
print(mesh, mesh_cropped)

o3d.visualization.draw_geometries([mesh_cropped], mesh_show_wireframe=True, mesh_show_back_face=True)


TriangleMesh with 152695 points and 305172 triangles. TriangleMesh with 49544 points and 98072 triangles.


In [8]:
# afterwork on the mesh
mesh_smooth = mesh_cropped.filter_smooth_taubin(number_of_iterations=3)
mesh_smooth.compute_triangle_normals()

TriangleMesh with 49544 points and 98072 triangles.

In [9]:
# writing the surface mesh to a file
output_file = 'testfiles/mesh_surface.ply'
o3d.io.write_triangle_mesh(output_file, mesh_smooth)


True

In [10]:
thickness = 20 #from lowest point of DEM to bottom of the output stl, standard value is calculated to be at sea level
# create a solid mesh from the surface mesh
solid_mesh = d3d.surface_to_volume(mesh_smooth, thickness)

# Visualize the updated mesh
o3d.visualization.draw_geometries([solid_mesh], mesh_show_wireframe=True, mesh_show_back_face=True)

# Export the extruded mesh to a file
output_file = 'testfiles/mesh_solid.stl'
o3d.io.write_triangle_mesh(output_file, solid_mesh) 



[Open3D WARNING] Ignoring attribute 'normals' for TensorMap with primary key 'positions'
[Open3D WARNING] Ignoring attribute 'colors' for TensorMap with primary key 'positions'
[Open3D WARNING] Ignoring attribute 'normals' for TensorMap with primary key 'indices'


True

# Buildings


In [11]:
### shrinking bbox so the buildings will fit proparly, but can result on slicing failures
shrink_factor = 0.001
bbox_product = d3d.shrink_bbox(bbox_product, shrink_factor)

In [12]:
### change to trimesh as it can cut through the middle of faces which open3d cannot
mesh_path = 'testfiles/3DStadtmodell_beschränkt.obj'
mesh = trimesh.load_mesh(mesh_path)

mesh_cut = d3d.cutting_mesh_through_middle_of_faces(mesh, bbox_product)
mesh_cut.show()


In [13]:
# Define edge_vertices
edge_vertices = mesh_cut.outline().vertices

### Filtering the vertices so that only edge points are left for each side
coordinates = np.array(edge_vertices[:, :3])

combined_mesh = mesh_cut

combined_mesh = d3d.process_missing_side_walls(coordinates, bbox_product, combined_mesh, 'east')
combined_mesh = d3d.process_missing_side_walls(coordinates, bbox_product, combined_mesh, 'west')
combined_mesh = d3d.process_missing_side_walls(coordinates, bbox_product, combined_mesh, 'north')
combined_mesh = d3d.process_missing_side_walls(coordinates, bbox_product, combined_mesh, 'south')

### trying to get more slice reliability
trimesh.repair.fill_holes(combined_mesh)

output_file = 'testfiles/stadtmodell_bbox.stl'
combined_mesh.export(output_file)

combined_mesh.show()

# testing
